In [40]:
import os
import sys

import numpy as np
#sys.path.insert(1, r"C:\Users\Zack\OneDrive\BCI\arl-eegmodels-master")
#from EEGModels import ShallowConvNet

# main directory
main_dir = os.getcwd()
# eeg data directory
eeg_dir = os.path.join(main_dir, "eeg_data")
# eeg validation data directory
eeg_validation_dir = os.path.join(main_dir, "eeg_data_validation")

MI_CLASSES = ["left", "right", "none"]

In [41]:
def to_one_sec(eeg_data, num_sec=5, samp_freq=250):
    one_sec_data = []
    for i in range(num_sec):
        eeg_data_i = eeg_data[i*samp_freq:(i+1)*samp_freq]
        one_sec_data.append(eeg_data_i)

    return one_sec_data

In [72]:
def create_data(data_dir):
    training_data_dict = {}

    for mi_class in MI_CLASSES:
        if mi_class not in training_data_dict:
            training_data_dict[mi_class] = []

        mi_class_dir = os.path.join(data_dir, mi_class)
        for filename in os.listdir(mi_class_dir):
            if "filtered" in filename:
                eeg_data = np.loadtxt(os.path.join(mi_class_dir, filename), delimiter=',')  # (250, 8) eeg data
                eeg_dataT = np.transpose(eeg_data)                                      # (8, 250) eeg data
                #one_sec_list = to_one_sec(eeg_data, num_sec=5, samp_freq=250)
                #for i in range(len(one_sec_list)):  # for each 1 second trial
                #    one_sec_data = one_sec_list[i] # (250, 8) eeg data
                #    one_sec_dataT = np.transpose(one_sec_data)  # (8, 250) eeg data
                #    training_data_dict[mi_class].append(one_sec_dataT)  
                training_data_dict[mi_class].append(eeg_dataT)  

    print("Trials per class: " + str(int(len(os.listdir(mi_class_dir))/3)) + "\n")
    session_count = [len(training_data_dict[mi_class]) for mi_class in MI_CLASSES]
    #print(f"One sec trial count:\nLeft: {session_count[0]}, Right: {session_count[1]}, None: {session_count[2]}\n")

    for mi_class in MI_CLASSES:
        np.random.shuffle(training_data_dict[mi_class])  # randomize session order
        training_data_dict[mi_class] = training_data_dict[mi_class][:min(session_count)]  # use min session count number of sessions

    # creating X, y 
    labeled_data = []
    for mi_class in MI_CLASSES:
        for data in training_data_dict[mi_class]:
            if mi_class == "left":
                labeled_data.append([data, [1, 0, 0]])
            elif mi_class == "right":
                labeled_data.append([data, [0, 1, 0]])
            elif mi_class == "none":
                labeled_data.append([data, [0, 0, 1]])

    np.random.shuffle(labeled_data)
    
    return labeled_data


In [73]:
labeled_data = create_data(data_dir=eeg_dir)

print("X shape: (num trials, num channels, num samples)")
print("y shape: (num trials, one hot labels)\n")

print("Creating labeled data...")
labeled_data_X = []
labeled_data_y = []

for X, y in labeled_data:
    labeled_data_X.append(X)
    labeled_data_y.append(y)
    
print("labeled_data_X: " + str(np.shape(labeled_data_X)))
print("labeled_data_y: " + str(np.shape(labeled_data_y)) + "\n")

print("Creating testing data...")
train_X = np.array(labeled_data_X[0:9*3])
train_y = np.array(labeled_data_y[0:9*3])
print("train_X: " + str(np.shape(train_X)))
print("train_y: " + str(np.shape(train_y)) + "\n")

print("Creating training data...")
test_X = np.array(labeled_data_X[9*3:12*3])
test_y = np.array(labeled_data_y[9*3:12*3])
print("test_X: " + str(np.shape(test_X)))
print("test_y: " + str(np.shape(test_y)) + "\n")

print("Creating validation data...")
validation_X = np.array(labeled_data_X[12*3:])
validation_y = np.array(labeled_data_y[12*3:])
print("validation_X: " + str(np.shape(validation_X)))
print("validation_y: " + str(np.shape(validation_y)) + "\n")

Trials per class: 15

X shape: (num trials, num channels, num samples)
y shape: (num trials, one hot labels)

Creating labeled data...
labeled_data_X: (45, 8, 1250)
labeled_data_y: (45, 3)

Creating testing data...
train_X: (27, 8, 1250)
train_y: (27, 3)

Creating training data...
test_X: (9, 8, 1250)
test_y: (9, 3)

Creating validation data...
validation_X: (9, 8, 1250)
validation_y: (9, 3)



In [61]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, AveragePooling2D
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, BatchNormalization
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

In [62]:
    def square(x):
        return K.square(x)

    def log(x):
        return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   

    def ShallowConvNet(nb_classes, Chans = 8, Samples = 250, dropoutRate = 0.5):
        # start the model
        input_main   = Input((Chans, Samples, 1))
        block1       = Conv2D(40, (1, 25), 
                                        input_shape=(Chans, Samples, 1),
                                        kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
        block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                                kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
        block1       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block1)
        block1       = Activation(square)(block1)
        block1       = AveragePooling2D(pool_size=(1, 75), strides=(1, 15))(block1)
        block1       = Activation(log)(block1)
        block1       = Dropout(dropoutRate)(block1)
        flatten      = Flatten()(block1)
        dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
        softmax      = Activation('softmax')(dense)

        return Model(inputs=input_main, outputs=softmax)

In [67]:
model = ShallowConvNet(nb_classes=3, Chans=8, Samples=250)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 8, 1250, 1)]      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 1226, 40)       1040      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 1226, 40)       12800     
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 1226, 40)       160       
_________________________________________________________________
activation_8 (Activation)    (None, 1, 1226, 40)       0         
_________________________________________________________________
average_pooling2d_2 (Average (None, 1, 77, 40)         0         
_________________________________________________________________
activation_9 (Activation)    (None, 1, 77, 40)         0   

In [68]:
model.compile(loss="categorical_crossentropy", optimizer ="adam", metrics=["accuracy"])

In [69]:
epochs = 50
batch_size = 10
fitted_model = model.fit(train_X, train_y, batch_size=batch_size, epochs=50, validation_data=(validation_X, validation_y))
#fitted_model = model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs, verbose=2)


Epoch 1/50
3/3 [==============================] - 1s 157ms/step - loss: 2.6329 - accuracy: 0.2917 - val_loss: 4.8172 - val_accuracy: 0.2222
Epoch 2/50
3/3 [==============================] - 0s 59ms/step - loss: 1.2235 - accuracy: 0.6463 - val_loss: 6.4491 - val_accuracy: 0.1111
Epoch 3/50
3/3 [==============================] - 0s 56ms/step - loss: 1.0858 - accuracy: 0.7144 - val_loss: 3.4486 - val_accuracy: 0.2222
Epoch 4/50
3/3 [==============================] - 0s 54ms/step - loss: 2.2722 - accuracy: 0.3722 - val_loss: 3.0750 - val_accuracy: 0.6667
Epoch 5/50
3/3 [==============================] - 0s 54ms/step - loss: 1.9762 - accuracy: 0.3727 - val_loss: 2.9135 - val_accuracy: 0.6667
Epoch 6/50
3/3 [==============================] - 0s 56ms/step - loss: 0.9291 - accuracy: 0.5653 - val_loss: 1.3661 - val_accuracy: 0.7778
Epoch 7/50
3/3 [==============================] - 0s 55ms/step - loss: 1.7621 - accuracy: 0.6954 - val_loss: 3.1382 - val_accuracy: 0.3333
Epoch 8/50
3/3 [==========

In [66]:
model = Sequential()

model.add(Conv1D(64, (3), input_shape=np.shape(train_X)))
model.add(Activation('relu'))

model.add(Conv1D(64, (2)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Conv1D(64, (2)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Flatten())

model.add(Dense(512))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

ValueError: Input 0 of layer max_pooling1d is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 27, 5, 64)